In [6]:
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation, rc
from IPython.display import HTML, Image # For GIF

rc('animation', html='html5')
np.random.seed(5)


# Set up formatting for the movie files
Writer = animation.writers['ffmpeg']
writer = Writer(fps=15, metadata=dict(artist='Me'), bitrate=1800)

class RiverFormation:
    """River formation model based off Kramer, Marder's paper "Evolution of river networks" 1993
    Land, and Water are maps of the same size that give, at each point, the height of soil, and the amount of water
    sitting on top of the soil respectively. The program loosely follows the following ruleset quoted from the above
    paper:
            (I) At each site of a lattice, we specify two integers, one corresponding to the height of land, the other to the height of water.
            (2) A lattice site is chosen randomly, and if the surface height (water plus land) is lower on a neighboring site, water units are moved to bring the surfaces as close to even as possible.
            (3) For each water unit transported out, a unit of land is dissolved away —but only if the land is lower at the destination site.
            (4) Additional water falls on a site as precipitation at random intervals.

    Parameters
    map_size (int, int): Sets the size of the corresponding land and water maps
    erosion_rate (float): Controls the rate of erosion
    land_initial_height (int): Initial height of the land plot
    precipitation (float): At each random precipitation site, this much watter is added
    land_condition (int): More of an enum to dictate the initial conditions of the land:
        0: Flat initial condition
        1: Random initial condition with randomness scaled by precipitation
        2: Sloping downward initial condition with slope set by precipitation
    """
    def __init__(self, map_size=(50, 25), erosion_rate=.1, land_initial_height=10, precipitation=0.1, land_condition=0):
        self.map_size = map_size
        self.erosion_rate = erosion_rate
        self.land_initial_height = land_initial_height
        self.precipitation = precipitation
        self.land_condition = land_condition
        
        # Chance of a map site being randomly chosen for rain
        self.precipitation_rate = 1 / map_size[0]

        # How many times to cycle through map and flow before another rainfall occurs
        #self.flow_per_rain = int(map_size[0] / 2)
        self.flow_per_rain = 10 # To speed up ...

        # Initially flat land (default)
        self.land = np.full(map_size, self.land_initial_height, dtype=float)
        self.water = np.zeros(map_size, dtype=float)
        self.flow_sum = np.zeros(map_size, dtype=float)

        # Initially uniform random land heights (scaled by precipitation)
        if self.land_condition == 1:
            self.land += np.random.uniform(-self.precipitation, self.precipitation, size=self.map_size)

        # Initially sloped land
        if self.land_condition == 2:
            column_deltas = self.precipitation * (np.arange(self.map_size[1]) + 1)
            self.land += column_deltas[np.newaxis, :]

    # Add some rain to sites randomly ...
    def add_precipitation(self):
        precipitation_sites = np.random.random(self.map_size) < self.precipitation_rate  # Boolean array (rain or no?)
        self.water[precipitation_sites] += self.precipitation # Add rain
        return np.argwhere(precipitation_sites)

    # Flow the water around to the neighbors and erode away. Find valid neighbors, sort them by who to flow to
    # first, then flow allotted ratio to each neighbor if acceptable.
    def flow(self, cx, cy):

        # We don't care about waterless sites
        if self.water[cx, cy] <= 0:
            return

        surface_height = self.land[cx, cy] + self.water[cx, cy]

        cy += 1 # prepare for padding
         
        # Do the padding
        left_pad_land = self.land[:, [0]] - self.precipitation
        right_pad_land = self.land[:, [-1]] + self.precipitation
        self.land = np.hstack((left_pad_land, self.land, right_pad_land))

        left_pad_water = np.zeros((self.map_size[0], 1))
        right_pad_water = np.zeros((self.map_size[0], 1))
        self.water = np.hstack((left_pad_water, self.water, right_pad_water))

        # Calculate neighbor positions. Use connected boundary for x.
        # For y we will have one boundary open for flow and one closed (as in the original paper).
        neighbors = np.array([
            [(cx - 1) % self.map_size[0], cy],  # Left
            [(cx + 1) % self.map_size[0], cy],  # Right
            [cx, cy + 1],  # Up
            [cx, cy - 1],  # Down
            [(cx - 1) % self.map_size[0], cy - 1],  # Left Down
            [(cx + 1) % self.map_size[0], cy - 1],  # Right Down
            [(cx - 1) % self.map_size[0], cy + 1],  # Left Up
            [(cx + 1) % self.map_size[0], cy + 1]  # Right Up
        ])

        # Try not to favor some direction when flowing ...
        np.random.shuffle(neighbors)

        # Separate neighbor x and y coordinates
        nx = neighbors[:, 0]
        ny = neighbors[:, 1]

        # Create a ranking system for how much water goes where. That way we have enough to go around to everyone, and it is 
        # proportional to the height difference.
        neighbor_surface_difference =((self.land[nx, ny] + self.water[nx, ny]) - surface_height)*(-1)
        neighbor_surface_difference[neighbor_surface_difference < 0] = 0
        height_diff_sum = np.sum(neighbor_surface_difference)
        neighbor_ratios = None
        if height_diff_sum > 0:
            neighbor_ratios = neighbor_surface_difference/height_diff_sum
        elif height_diff_sum ==0:
            neighbor_ratios = [.125, .125, .125, .125, .125, .125, .125, .125]
        else:
            print("Ratio error")

        for i in range(len(neighbors[0])):
            # Check if neighbor is within bounds
            nx = neighbors[i][0]
            ny = neighbors[i][1]
            ratio = neighbor_ratios[i]
            neighbor_in_bounds = 1 <= ny <= self.map_size[1] # Dont count padded sites
            neighbor_land_height = self.land[nx, ny]
            neighbor_surface_height = self.land[nx, ny] + self.water[nx, ny]
            neighbor_water_height = self.water[nx, ny]

            # Proceed if water can flow to the neighbor
            if neighbor_surface_height < surface_height and self.water[cx, cy] > 0:
                
                flow_amount = 0
                
                # Case 1: Land is higher than neighbor
                if self.land[cx, cy] > neighbor_land_height: 
                    #Case 1a - there's enough water to fill the difference
                    if (neighbor_water_height+self.water[cx, cy]) < (self.land[cx, cy] - neighbor_land_height): 
                        flow_amount = ratio*self.water[cx, cy]
                        flow_amount -= flow_amount * self.erosion_rate / 2
                    # Case 2a - Not enough water to fill the difference
                    elif (neighbor_water_height+self.water[cx, cy]) > (self.land[cx, cy] - neighbor_land_height):
                        flow_amount = ratio*(self.water[cx, cy] - ((self.water[cx, cy] + neighbor_water_height) -
                                                                   (self.land[cx, cy] - neighbor_land_height))/2)
                        flow_amount -= flow_amount * self.erosion_rate / 2

                else: # Case 2: The neighbor's land is lower than ours, but its water is higher than our land
                    flow_amount = min(self.water[cx, cy], ratio*((surface_height - neighbor_surface_height)/2))
                    flow_amount -= flow_amount * self.erosion_rate / 2

                # Update water levels at current site
                self.water[cx, cy] -= flow_amount
                self.flow_sum[cx, cy-1] += flow_amount

                if neighbor_in_bounds:
                    # Neighbor is within bounds, update water there
                    self.water[nx, ny] += flow_amount

                # Erode land at the source site if the land at the destination is lower
                self.land[cx, cy] -= (self.erosion_rate * flow_amount)
        
        # Get rid of padding
        self.water = self.water[:, 1:-1]
        self.land = self.land[:, 1:-1]


    def step(self):
        # Perform precipitation
        self.add_precipitation()
        
        # Randomize where the flow starts from
        xs = list(range(self.map_size[0]))
        ys = list(range(self.map_size[1]))
        np.random.shuffle(xs)
        np.random.shuffle(ys)
        
        # Flow water starting from each precipitation site
        for _ in range(self.flow_per_rain):
            for x in xs:
                for y in ys:
                    self.flow(x, y)
    
    def run(self, iterations):
        # Initialize lists to store states
        land_states = []
        flow_sum_states = []

        # Run the simulation for a number of iterations
        for i in tqdm(range(iterations)):
            self.step()

            # Store the states at each iteration
            land_states.append(self.land.copy())
            flow_sum_states.append(self.flow_sum.copy())

        # Calculate global min/max values for both datasets
        land_min = min(frame.min() for frame in land_states)
        land_max = max(frame.max() for frame in land_states)
        flow_min = min(frame.min() for frame in flow_sum_states)
        flow_max = max(frame.max() for frame in flow_sum_states)

        # Create the initial plot with subplots
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))
        
        # Initial plots with fixed color ranges
        im1 = ax1.imshow(land_states[0], cmap='terrain', 
                        vmin=land_min, vmax=land_max)
        im2 = ax2.imshow(flow_sum_states[0], cmap='viridis',
                        vmin=flow_min, vmax=flow_max)
        
        # Add colorbars
        plt.colorbar(im1, ax=ax1, label='Elevation')
        plt.colorbar(im2, ax=ax2, label='Flow Accumulation')
        
        # Set titles
        ax1.set_title('Terrain')
        ax2.set_title('Flow Accumulation')

        # Animation update function
        def update(frame):
            # Update both plots (color range is already fixed)
            im1.set_array(land_states[frame])
            im2.set_array(flow_sum_states[frame])
            return [im1, im2]

        # Create the animation object
        anim = animation.FuncAnimation(
            fig, 
            update,
            frames=len(land_states),
            interval=30,  # 30ms between frames
            blit=True,
            repeat=True
        )

        plt.close() # Prevents display of static plot
        
        # Save as HTML5 video
        html = HTML(anim.to_jshtml())
        
        # Save as GIF if needed
        anim.save('ani.gif', writer='pillow', fps=30)
        
        # Display the animation in the notebook
        display(html)

# Set size of image here
basin = RiverFormation((50, 20), erosion_rate=0.2, land_initial_height=0, precipitation=.1, land_condition=0)

# Set Number of iterations here
basin.run(iterations=200)

ModuleNotFoundError: No module named 'tqdm'